In [29]:
import random
import pandas as pd
import numpy as np
from scipy.stats import norm

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
from SyncRNG import SyncRNG
import numpy as np
import re
from statsmodels.sandbox.stats.multicomp import multipletests
from scipy import linalg
from itertools import chain

from SyncRNG import SyncRNG

from CTL.causal_tree_learn import CausalTree
from sklearn.model_selection import train_test_split
import plotnine as p
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [10]:
data = pd.read_csv( "https://raw.githubusercontent.com/grf-labs/grf/master/experiments/acic18/synthetic_data.csv" )
data

,schoolid,Z,Y,S3,C1,C2,C3,XC,X1,X2,X3,X4,X5
0,76,1,0.081602,6,4,2,1,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
1,76,1,-0.385869,4,12,2,1,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
2,76,1,0.398184,6,4,2,0,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
3,76,1,-0.175037,6,4,2,0,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
4,76,1,0.884583,6,4,1,0,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10386,1,0,0.423366,7,4,2,1,3,1.185986,-1.129889,1.009875,1.005063,-1.174702
10387,1,0,-0.197092,7,4,2,1,3,1.185986,-1.129889,1.009875,1.005063,-1.174702
10388,1,0,0.141698,2,15,1,1,3,1.185986,-1.129889,1.009875,1.005063,-1.174702
10389,1,0,-0.351565,5,4,1,1,3,1.185986,-1.129889,1.009875,1.005063,-1.174702


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10391 entries, 0 to 10390
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   schoolid  10391 non-null  int64  
 1   Z         10391 non-null  int64  
 2   Y         10391 non-null  float64
 3   S3        10391 non-null  int64  
 4   C1        10391 non-null  int64  
 5   C2        10391 non-null  int64  
 6   C3        10391 non-null  int64  
 7   XC        10391 non-null  int64  
 8   X1        10391 non-null  float64
 9   X2        10391 non-null  float64
 10  X3        10391 non-null  float64
 11  X4        10391 non-null  float64
 12  X5        10391 non-null  float64
dtypes: float64(6), int64(7)
memory usage: 1.0 MB


In [12]:

data.schoolid = data.schoolid.astype('category')


In [14]:
# select columns using positions

data.iloc[:,1:13]

,Z,Y,S3,C1,C2,C3,XC,X1,X2,X3,X4,X5
0,1,0.081602,6,4,2,1,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
1,1,-0.385869,4,12,2,1,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
2,1,0.398184,6,4,2,0,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
3,1,-0.175037,6,4,2,0,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
4,1,0.884583,6,4,1,0,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
...,...,...,...,...,...,...,...,...,...,...,...,...
10386,0,0.423366,7,4,2,1,3,1.185986,-1.129889,1.009875,1.005063,-1.174702
10387,0,-0.197092,7,4,2,1,3,1.185986,-1.129889,1.009875,1.005063,-1.174702
10388,0,0.141698,2,15,1,1,3,1.185986,-1.129889,1.009875,1.005063,-1.174702
10389,0,-0.351565,5,4,1,1,3,1.185986,-1.129889,1.009875,1.005063,-1.174702


In [18]:
# Compute school.id variable
data.schoolid = data.schoolid.astype('float')



In [41]:
data.schoolid.unique()

array([76., 75., 74., 73., 72., 71., 70., 69., 68., 67., 66., 65., 64.,
       63., 62., 61., 60., 59., 58., 57., 56., 55., 54., 53., 52., 51.,
       50., 49., 48., 47., 46., 45., 44., 43., 42., 41., 40., 39., 38.,
       37., 36., 35., 34., 33., 32., 31., 30., 29., 28., 27., 26., 25.,
       24., 23., 22., 21., 20., 19., 18., 17., 16., 15., 14., 13., 12.,
       11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.])

In [ ]:

# Dummys variables are formed from the variable schoolid

In [46]:
data[[f"var_{i}" for i in range(1,77)]] = pd.get_dummies(data.schoolid) 

In [51]:
schoolmat=data[[f"var_{i}" for i in range(1,77)]]

In [53]:
schoolmat

,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10386,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10387,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10388,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10389,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
# Number of observations that each school.id variable has
schoolsize=np.size(schoolmat)

In [56]:

school.size

789716